## Assignment

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Use the Geocoder package or the csv file to create the following dataframe:

Important Note: There is a limit on how many times you can call geocoder.google function. It is 2500 times per day. This should be way more than enough for you to get acquainted with the package and to use it to get the geographical coordinates of the neighborhoods in the Toronto.

Once you are able to create the above dataframe, submit a link to the new Notebook on your Github repository. (2 marks)

In [7]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda update -n base -c defaults conda -y # update Conda

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge
    sci

<a id='item1'></a>

## 1. Scrape Wiki Page List of Toronto Neighborhoods

Use beautifulsoup to scrape Wiki (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)

In [2]:
!pip install beautifulsoup4

import urllib.request
import urllib.parse
import urllib.error
from bs4 import BeautifulSoup
import ssl

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

check = False
while check is False:
    try:
        check = True
        html = urllib.request.urlopen(url).read()
    except:
        check = False
        print("Error")
        url = input("Enter site: ")
    continue

soup = BeautifulSoup(html, "html.parser")

#print(soup.prettify())

# Retrieve all of the anchor tags
tags = soup('td')

scraped_data = list()
for tag in tags:
    try:
        #print('TAG:', tag.contents[0].string)
        word = str(tag.contents[0].string)
        scraped_data.append(word)
            
    except: break

count=0 
postal_code = list()
borough = list()
neighborhood = list()
for item in scraped_data:
    if count==0: 
        postal_code.append(item)
        count=1
    elif count==1:
        borough.append(item)
        count=2
    elif count==2:
        item = item.rstrip()
        neighborhood.append(item)
        count=0

#print(postal_code)
#print(borough)
#print(neighborhood)


zipped_list = list(zip(postal_code, borough, neighborhood))
#zipped_list

In [3]:
import pandas as pd
import re

neighborhoods = pd.DataFrame(zipped_list, columns = ['Postal Code' , 'Borough', 'Neighborhood']) 

# Drop rows if borough is 'Not assigned'. Reset index.

index_bor = neighborhoods[ neighborhoods['Borough'] == 'Not assigned' ].index
 
neighborhoods.drop(index_bor, inplace=True)

neighborhoods.reset_index(drop=True, inplace=True)

# Replace neighborhood name with name of borough if neighborhood not assigned

neighborhoods['duplicates'] = pd.Series([0 for x in range(len(neighborhoods.index))], index=neighborhoods.index)

count=0
for item in neighborhoods['Neighborhood']:
    #print(item)
    if item=='Not assigned':
        neighborhoods['Neighborhood'][count] = neighborhoods['Borough'][count]
    
    try:
        if neighborhoods['Postal Code'][count] == neighborhoods['Postal Code'][count+1]:
            newvar = neighborhoods['Neighborhood'][count] + ', ' + neighborhoods['Neighborhood'][count+1]
            neighborhoods['Neighborhood'][count] = newvar
            neighborhoods['duplicates'][count+1] = 1
    except: continue
    
    count=count+1

# Drop duplicate Postal Codes

new_index = neighborhoods[ neighborhoods['duplicates'] == 1 ].index
neighborhoods.drop(new_index , inplace=True)
neighborhoods.reset_index(drop=True, inplace=True)
neighborhoods.drop(['duplicates'], axis=1, inplace=True)

#neighborhoods.head(20)


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

neighborhoods.shape


The dataframe has 11 boroughs and 210 neighborhoods.


(210, 3)

#### Import geospatial data for each postal code and merge with dataframe.

In [4]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url2 = 'https://cocl.us/Geospatial_data'

check = False
while check is False:
    try:
        check = True
        html = urllib.request.urlopen(url).read()
    except:
        check = False
        print("Error")
        url = input("Enter site: ")
    continue
    

geodf = pd.read_csv(url2)
geodf.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
torontodf = pd.merge(neighborhoods, geodf, how='inner', on='Postal Code', left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

torontodf.sort_values(by=['Latitude'], inplace=True)
torontodf.reset_index(drop=True, inplace=True)

torontodf.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
1,M8V,Etobicoke,"Humber Bay Shores, Mimico South",43.605647,-79.501321
2,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW",43.628841,-79.520999
3,M5V,Downtown Toronto,"CN Tower, Bathurst Quay",43.628947,-79.394420
4,M8Y,Etobicoke,"Humber Bay, King's Mill Park",43.636258,-79.498509
5,M6K,West Toronto,"Brockton, Exhibition Place",43.636847,-79.428191
6,M7R,Mississauga,Canada Post Gateway Processing Centre,43.636966,-79.615819
7,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands",43.640816,-79.381752
8,M9C,Etobicoke,"Bloordale Gardens, Eringate",43.643515,-79.577201
9,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
